COVID-19 cases per English Local Authority 
==

Data taken from https://coronavirus.data.gov.uk/

In [ ]:
#https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/populationestimatesforukenglandandwalesscotlandandnorthernireland


!wget -N -P data "https://coronavirus.data.gov.uk/downloads/csv/coronavirus-cases_latest.csv"
!wget -nc -O data/population.xlsx "https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fpopulationandmigration%2fpopulationestimates%2fdatasets%2fpopulationestimatesforukenglandandwalesscotlandandnorthernireland%2fmid2019april2020localauthoritydistrictcodes/ukmidyearestimates20192020ladcodes.xls"


In [ ]:
import warnings
import pandas as pd
import numpy as notebook
import matplotlib.pyplot as plt
import matplotlib as mp
import time
import geopandas
import ipywidgets as widgets
from IPython.core.display import display, HTML
warnings.filterwarnings('ignore')

In [ ]:
#read in the ONS population spreadsheet
df_pop = pd.read_excel("data/population.xlsx", sheet_name=5,header=4)

#read in the coronaviruses csv
df_cases = pd.read_csv("data/coronavirus-cases_latest.csv", index_col=None, header=0)
    
#create a proper date field
df_cases['date'] = pd.to_datetime(df_cases['Specimen date'])
df_cases['date-index'] = pd.to_datetime(df_cases['Specimen date'])
df_cases.set_index('date-index', inplace=True)

df_cases = df_cases[df_cases['date'] > pd.to_datetime('2020-03-01')]

#merge ons and cases data
df = pd.merge(df_cases, df_pop, how='inner', left_on='Area code', right_on='Code')

#create rate col for daily lab confirmed cases
df["Daily lab-confirmed cases rate"] = (df["Daily lab-confirmed cases"] / df["All ages"])*100000

In [ ]:
def compare_regions(region_1_stats, region_2_stats):

    plt.style.use('seaborn-white')

    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.serif'] = 'Arial'
    plt.rcParams['font.monospace'] = 'Ubuntu Mono'
    plt.rcParams['font.size'] = 20
    plt.rcParams['axes.labelsize'] = 14
    plt.rcParams['axes.labelweight'] = 'normal'
    plt.rcParams['axes.titlesize'] = 16
    plt.rcParams['xtick.labelsize'] = 12
    plt.rcParams['ytick.labelsize'] = 12
    plt.rcParams['legend.fontsize'] = 14
    plt.rcParams['figure.titlesize'] = 18
    plt.rcParams['lines.linewidth'] = 3
    
    region_1_stats["7-day-rolling-average"] = region_1_stats["Daily lab-confirmed cases rate"].rolling(10, min_periods=1).mean()
    region_2_stats["7-day-rolling-average"] = region_2_stats["Daily lab-confirmed cases rate"].rolling(10, min_periods=1).mean()
    
    region_1_name = region_1_stats['Name'].max()
    region_2_name = region_2_stats['Name'].max()
    
    latest_figures = region_1_stats[region_1_stats['date']==region_1_stats['date'].max()]
    
    #display((latest_figures['Daily lab-confirmed cases']))

    #display(HTML('<h3>Total number of lab confirmed cases</h3>' + latest_figures['Cumulative lab-confirmed cases'].to_string()))
    
    #this_la_boundary = local_authority_boundaries.merge(latest_figures, how='right', left_on='CODE', right_on='Area code')
    #display(this_la_boundary.head())
    #a = geopandas.sjoin(this_la_boundary, local_authority_boundaries, how="inner", op='intersects')
    #a.plot()
    
    
    fig, (ax1, ax2) = plt.subplots(2,1,sharex=True,figsize=(15,15))
        
    fig.suptitle('Comparing ' + region_1_name + ' and ' + region_2_name, ha='left',va='bottom')
    
    ax1.plot_date(region_1_stats["date"],
                 region_1_stats["7-day-rolling-average"],
                 linestyle='solid',
                 color="#118ab2",
                 ms=0
                )
    
    ax1.plot_date(region_2_stats["date"],
                 region_2_stats["7-day-rolling-average"],
                 linestyle='solid',
                 color="#ef476f",
                 ms=0
                )
    
    ax1.axvline(pd.to_datetime(['3/23/2020']), linestyle="dotted", color="#f9c74f")
    ax1.axvline(pd.to_datetime(['5/10/2020']), linestyle="dotted", color="#f8961e" )    
    ax1.axvline(pd.to_datetime(['6/1/2020']), linestyle="dotted", color="#f3722c" )                         


    ax1.set_title('Lab-confirmed cases per 100k population (7 day rolling average)',loc="left")  
    
    ax1.legend(labels=[region_1_name,
                       region_2_name,
                       'Lockdown start',
                       'Stay Alert phase start',
                       'Phase 2 lockdown start' ],
               frameon=True )
    
    plt.xticks(rotation=45);  
    ax1.spines['top'].set_visible(False)
    ax1.spines['bottom'].set_visible(False)
    ax1.spines['left'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.grid(True,which='major',axis='both')
    
    #ax1.plot()
    ax2.plot_date(region_1_stats["date"],
                 region_1_stats["Cumulative lab-confirmed cases rate"],
                 linestyle='solid',
                 color="#118ab2",
                 ms=0)
    
    ax2.plot_date(region_2_stats["date"],
                 region_2_stats["Cumulative lab-confirmed cases rate"],
                 linestyle='solid',
                 color="#ef476f",
                 ms=0)
    
    ax2.axvline(pd.to_datetime(['3/23/2020']), linestyle="dotted", color="#f9c74f")
    ax2.axvline(pd.to_datetime(['5/10/2020']), linestyle="dotted", color="#f8961e" )    
    ax2.axvline(pd.to_datetime(['6/1/2020']), linestyle="dotted", color="#f3722c" )                         

    ax2.set_title('Cumulative lab-confirmed cases per 100k population',loc='left')
   
    plt.xticks(rotation=60);  
    ax2.spines['top'].set_visible(False)
    ax2.spines['bottom'].set_visible(False)
    ax2.spines['left'].set_visible(False)
    ax2.spines['right'].set_visible(False) 
    ax2.grid(True,which='major',axis='both')
    
    #ax2.plot()
    
    return fig
    

In [ ]:
#local_authority_boundaries = geopandas.read_file('./data/Data/GB/district_borough_unitary_region.shp')

#local_authority_boundaries.head()
all_areas = df['Area name'].unique()

dropdown_area_1 = widgets.Dropdown(options=all_areas,value=all_areas.min(), description="Compare")
dropdown_area_2 = widgets.Dropdown(options=all_areas,value="England", description="with")

output_stats = widgets.Output()

def update_graphs():
    region_1_stats = df[df['Area name']==dropdown_area_1.value]
    region_2_stats = df[df['Area name']==dropdown_area_2.value]
    output_stats.clear_output()
    with output_stats:
        display(compare_regions(region_1_stats, region_2_stats))
        

def dropdown_area_1_eventhandler(change):
    update_graphs()
    
def dropdown_area_2_eventhandler(change):
    update_graphs()
    
        
dropdown_area_1.observe(dropdown_area_1_eventhandler, names='value')
dropdown_area_2.observe(dropdown_area_2_eventhandler, names='value')
display(dropdown_area_1,dropdown_area_2,output_stats)
update_graphs()